In [ ]:
import os
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Adding AWS S3 Minio configs
sparkConf = (
    SparkConf()
    .set("spark.jars.ivy","/home/brijeshdhaker/.ivy2")
    .set("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .set("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.0.0,io.delta:delta-spark_2.12:3.3.2")
    .set("spark.executor.heartbeatInterval", "300000")
    .set("spark.network.timeout", "400000")
    .set("spark.hadoop.fs.defaultFS", "s3a://defaultfs/")
    .set("spark.hadoop.fs.s3a.endpoint", "http://minio.sandbox.net:9010")
    .set("spark.hadoop.fs.s3a.access.key", "pgm2H2bR7a5kMc5XCYdO")
    .set("spark.hadoop.fs.s3a.secret.key", "zjd8T0hXFGtfemVQ6AH3yBAPASJNXNbVSx5iddqG")
    .set("spark.hadoop.fs.s3a.path.style.access", "true")
    .set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    #.set("spark.eventLog.enabled", "true")
    #.set("spark.eventLog.dir", "file:///apps/var/logs/spark-events")
)

spark = (
    SparkSession.builder.master("local[*]").
        appName('spark-deltalake').
        config(conf=sparkConf).
        getOrCreate()
)

spark.sparkContext.setLogLevel('ERROR')
spark

#
# 
#
def display(df):
    df.show(truncate=False)

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType

schema = StructType([
  StructField("VendorID", LongType(), True),
  StructField("tpep_pickup_datetime", TimestampNTZType(), True),
  StructField("tpep_dropoff_datetime", TimestampNTZType(), True),
  StructField("passenger_count", DoubleType(), True),
  StructField("trip_distance", DoubleType(), True),
  StructField("RatecodeID", DoubleType(), True),
  StructField("store_and_fwd_flag", StringType(), True),
  StructField("PULocationID", LongType(), True),
  StructField("DOLocationID", LongType(), True),
  StructField("payment_type", LongType(), True),
  StructField("fare_amount", DoubleType(), True),
  StructField("extra", DoubleType(), True),
  StructField("mta_tax", DoubleType(), True),
  StructField("tip_amount", DoubleType(), True),
  StructField("tolls_amount", DoubleType(), True),
  StructField("improvement_surcharge", DoubleType(), True),
  StructField("total_amount", DoubleType(), True),
  StructField("congestion_surcharge", DoubleType(), True),
  StructField("airport_fee", DoubleType(), True)
])

df = spark.read.format("parquet").load("s3a://datasets/taxi-data/yellow_tripdata_2021-04.parquet")
df.printSchema()
#
display(df)



# Create the table if it does not exist. Otherwise, replace the existing table.
#df.writeTo("peoples").createOrReplace()

# If you know the table does not already exist, you can call this instead:
# df.write.saveAsTable("main.default.people_10m")